    This file more or less describes the timeline of problems that we experienced in our case study attempts. 
    To easily show these problems, all errors are printed and explained in this notebook. 

    The first cell block imports all packages and files that are required through all attempts. 

In [1]:
import sys
sys.path.append("../..")
import numpy as np
import pandas as pd
import geopandas as gpd
from dyntapy.assignments import StaticAssignment
from dyntapy.toll import create_toll_object
from dyntapy.demand_data import od_graph_from_matrix
from pickle import load 
import warnings
warnings.filterwarnings('ignore') # hide warnings

network_path = "C:/Users/anton/IP2/toll_optimization/data_map/HEEDS_input/network_with_centroids/elastic_BRUSSEL_40"
od_path = "C:/Users/anton/IP2/toll_optimization/data_map/HEEDS_input/od_graph/elastic_BRUSSEL_40.xlsx"
A_matrix_path = "C:/Users/anton/IP2/toll_optimization/data_map/HEEDS_input/elastic/A_matrix_BRUSSEL_40"
B_matrix_path = "C:/Users/anton/IP2/toll_optimization/data_map/HEEDS_input/elastic/B_matrix_BRUSSEL_40"
x_centroids_path = "C:/Users/anton/IP2/toll_optimization/data_map/HEEDS_input/elastic/x_centroids_BRUSSEL_40"
y_centroids_path = "C:/Users/anton/IP2/toll_optimization/data_map/HEEDS_input/elastic/y_centroids_BRUSSEL_40"

with open(network_path, 'rb') as network_file:
    g = load(network_file)
with open(A_matrix_path, 'rb') as matrix:
    A = load(matrix)
with open(B_matrix_path, 'rb') as matrix:
    B = load(matrix)
old_od = pd.read_excel(od_path)
old_od = old_od.to_numpy()
x_centroids = np.loadtxt(x_centroids_path)
y_centroids = np.loadtxt(y_centroids_path)
original_od_graph = od_graph_from_matrix(old_od,x_centroids,y_centroids) 

-----
    FIRST ATTEMPT: TOLLING BASED ON OUR DEFINED CORDON OUTSIDE RING OF BRUSSELS

    The ring cordon that we defined consists of 115 links, which are shown in data_map/images/ring_cordon.png. The links highlighted in red are the links that will be tolled. 

    Running an STA on this cordon fails quickly. Further discussed below the cell block.

In [2]:
toll_method = 'cordon'
toll_value = 0.05

# Load links for cordon outside ring of Brussels. 
links_path = "C:/Users/anton/IP2/toll_optimization/data_map/QGIS/ring_cordon/links_crossing_cordon.shp"
links_crossing_cordon = gpd.read_file(links_path)
toll_link_ids = [toll_id for toll_id in links_crossing_cordon['link_id']]

toll_object = create_toll_object(g, toll_method, toll_link_ids, toll_value)
assignment = StaticAssignment(g, original_od_graph, toll_object)
result = assignment.run('dial_b')

init passed successfully
initial loading starts 


AssertionError: 

As you can see in the printed assertion error above, there is, at some point in the dial-b algorithm, a minimum path flow that is negative. We don't know what is causing this, nor what we can do about it. It's complicated for us to diagnose why this is happening, because in the first dial-b iterations nothing is wrong. Furthermore, it's not the easiest piece of code for us to understand, as it's quite complex. 

Nevertheless, we do know that that tolling a single link in the Brussels network did not cause any errors. We also know that tolling multiple links in the toy networks did not cause errors either (see elastic_toy_STA.ipynb). So it is possible to toll multiple links, however it fails for our cordon of 115 links. 

As we did not know what was causing the problem, it was not possible for us to resolve it and continue working with this defined cordon. We also don't know what the tipping point is to cause the errors (small number of links works, large number does not). Therefore, we decided to try a case study on a smaller cordon. (from 115 links to 48, see next attempt)

-----
    SECOND ATTEMPT: TOLLING BASED ON THE DEFINED PENTAGON CORDON

    The pentagon cordon that we defined consists of 48 links, which is shown in data_map/images/pentagon_cordon.png. The links highlighted in red are the links that will be tolled. 

In [3]:
toll_method = 'cordon'
toll_value = 0.05

# Load links for pentagon defined cordon. 
links_path = "C:/Users/anton/IP2/toll_optimization/data_map/QGIS/pentagon/links_crossing_pentagon.shp"
links_crossing_cordon = gpd.read_file(links_path)
toll_link_ids = [toll_id for toll_id in links_crossing_cordon['link_id']]

toll_object = create_toll_object(g, toll_method, toll_link_ids, toll_value)
assignment = StaticAssignment(g, original_od_graph, toll_object)
result = assignment.run('dial_b')

init passed successfully
initial loading starts 


AssertionError: 

As you can see, the same assertion error occurs for a cordon with only 48 links. At this point, it was decided together with the assistant to limit our case study to a case where we would manually select a couple of links. This brings us to attempt 3, where we manually selected some links which would still make sense to toll.

------
    THIRD ATTEMPT: TOLLING THE E19- AND E40-JUNCTION TO THE RING OF BRUSSELS

    We selected 15 links related to the E19- and E40-junction to toll. These are shown in red in data_map/images/E19_and_E40_junctions.png.

In [4]:
toll_method = 'cordon'
toll_value = 0.05
toll_link_ids = [642, 654, 760, 939, 1347, 1354, 1367, 2970, 6355, 6724, 10692, 10693, 15600, 15845, 15848]

toll_object = create_toll_object(g, toll_method, toll_link_ids, toll_value)
assignment = StaticAssignment(g, original_od_graph, toll_object)
result = assignment.run('dial_b')

init passed successfully
initial loading starts 


AssertionError: 

As you can see, the same error occurs again. We still don't know what is causing this, but we can see that it took way longer for this error to occur, which means that we are deeper into the dial-B algorithm. 

-----
    FOURTH ATTEMPT: TOLLING THE E19-JUNCTION TO THE RING OF BRUSSELS

    Compared to attempt three, this attempt only uses the 6 links related to the E19-junction. These are shown in red in data_map/images/E19_junction.png.

    Again, we tried to run the STA on the selected links. Finally, this was successfull, as you can see below. 

In [15]:
toll_method = 'cordon'
toll_value = 0.1
toll_link_ids = [939, 1347, 2970, 6355, 6724, 15600]

toll_object = create_toll_object(g, toll_method, toll_link_ids, toll_value)
assignment = StaticAssignment(g, original_od_graph, toll_object)
result = assignment.run('dial_b')

init passed successfully
initial loading starts 


We then took this case to Heeds, but barely any improvements were made which sounded weird to us. That's why we decided to critically review whether everything is correct. The remainder of this notebook explains how we tried to check that, and what is going wrong. 

In the elastic demand case, we modify our OD based on the new skims (because the tolls are changing the skims for some OD-paths). The next cell blocks shows these modifications.

In [7]:
new_od = (A-result.skim)/B
old_od
print("Biggest change occuring across the OD matrix for one specific OD:", np.max(abs((new_od - old_od))))
print("Relative Frobenius norm change of the OD matrix:", abs((np.linalg.norm(new_od) - np.linalg.norm(old_od))/np.linalg.norm(old_od)))

Biggest change occuring across the OD matrix for one specific OD: 0.1725614748217843
Relative Frobenius norm change of the OD matrix: 8.986609781978718e-09


So we see that there is hardly any change in our OD-matrix. The largest change for a specific OD-pair is only 0,2 passengers. There are 2 things that could cause this:
    
- It could be that the tolled links have a good alternative, such that the skims of each OD does not really change. 
    
- It could be that there is an implementation problem, which causes the users to not feel the toll. Therefore, they would not have to reroute and there would be no change in list costs. 

Ofcourse, we must verify that we are not in the second case. The first thing that we need to to evaluate the implementation correctness, is an STA without tolls. This allows us to compare the STA with toll to a reference case. 
    
Therefore, the following cell block runs an STA on our network and OD-matrix without tolls. 

In [6]:
assignment_no_toll = StaticAssignment(g, original_od_graph)
result_no_toll = assignment_no_toll.run('dial_b')

init passed successfully
initial loading starts 


So we now have STA results for a situation with toll and without toll. We can now verify whether the link costs and link flows of the tolled links change correctly subject to the charged toll. 

In [18]:
# Case of no toll: 
flows_no_toll = result_no_toll.flows
costs_no_toll = result_no_toll.link_costs
for link_id in toll_link_ids:
    print("flow on link", link_id, ":", flows_no_toll[link_id])
    print("cost on link", link_id, ":", costs_no_toll[link_id])

nr_links_higher_costs_than_toll = 0
for link in range(len(costs_no_toll)):
    if costs_no_toll[link] > 0.05:
        nr_links_higher_costs_than_toll += 1
print("number of links with a cost higher than the set toll value:", nr_links_higher_costs_than_toll)


flow on link 939 : 7716.673868033565
cost on link 939 : 0.005215132800393459
flow on link 1347 : 5642.645000802237
cost on link 1347 : 0.031246173716437634
flow on link 2970 : 5028.34600417281
cost on link 2970 : 0.005048017454873437
flow on link 6355 : 151.67900016251951
cost on link 6355 : 0.005351705134362368
flow on link 6724 : 5348.786001697648
cost on link 6724 : 0.026260507392172044
flow on link 15600 : 859.1819982184097
cost on link 15600 : 0.005357100447114171
number of links with a cost higher than the set toll value: 443


In [19]:
# Case of toll:
flows_toll = result.flows
costs_toll = result.link_costs
for link_id in toll_link_ids:
    print("flow on link", link_id, ":", flows_toll[link_id])
    print("cost on link", link_id, ":", costs_toll[link_id])

nr_links_higher_costs_than_toll = 0
for link in range(len(costs_toll)):
    if costs_toll[link] > 0.05:
        nr_links_higher_costs_than_toll += 1
print("number of links with a cost higher than the set toll value:", nr_links_higher_costs_than_toll)

flow on link 939 : 7715.1327282452
cost on link 939 : 0.005214922464603186
flow on link 1347 : 5657.848000827129
cost on link 1347 : 0.03128167900518951
flow on link 2970 : 5080.005004390725
cost on link 2970 : 0.005050018401972755
flow on link 6355 : 0.0
cost on link 6355 : 0.10535169988870621
flow on link 6724 : 5346.956001833547
cost on link 6724 : 0.026257399009465023
flow on link 15600 : 855.2409980548546
cost on link 15600 : 0.00535700203904884
number of links with a cost higher than the set toll value: 443


As you can see based on the output from the previous two cell blocks, there is no significant change in flow or link cost for the tolled links, except for link 6355. We expected all links to have costs and flows similar to link 6355. Namely, the 0.1 toll value that was charged equals 6 minutes of travel time. However, these tolled links don't show this in their link cost, nor is their flow affected. This is really weird, because the link costs for tolled links can NEVER be lower than the charged toll value plus the free flow cost. The only explanation that we could see for this is implementation mistakes. However, we think that the implementation mistake is not in our hands. If our tolling implementation was to be the problem, you'd expect to see incorrect behaviour FOR ALL tolled links, and not for a subset. 

Furthermore, we think our tolling implementation is correct given that tolling multiple links in the toy network succeeded. Therefore, we would conclude that our tolling implementation works as intended.  